### Testing Data Helpers

In [ ]:
from MessagesDB import MessagesDB

In [ ]:
from data_helpers import get_messages

In [ ]:
messages_db = MessagesDB()

In [ ]:
messages = get_messages(messages_db)

In [ ]:
len(messages)